### Introduction

In this project, we'll be using linear regression models to try to predict a day's closing stock price. The dataset we'll be using contains information on the S&P 500 Index and so contains only aggregate data on 500 of the US's 500 large cap companies (those with market capitalization (individual stock price x number of shares) greater than $5 billion.

The dataset contains information on the S&P 500 from 1950 up to 2015. We'll be using data from before 2013 to train our model, and then the rest of the data to test our model.

As for the information itself, the dataset mostly contains information on stock prices for each trading day. It contains the daily opening, high, low, and closing prices, the adjusted close price, and the volume of trades that occurred on that day. We'll only be using some of this information to predict the close price, but we'll get to this in a second.

In [101]:
import pandas as pd
from datetime import datetime

data = pd.read_csv("sphist.csv")
data["Date"] = pd.to_datetime(data["Date"])
data = data.sort_values(by = ["Date"])
print(data.head())

            Date   Open   High    Low  Close     Volume  Adj Close
16589 1950-01-03  16.66  16.66  16.66  16.66  1260000.0      16.66
16588 1950-01-04  16.85  16.85  16.85  16.85  1890000.0      16.85
16587 1950-01-05  16.93  16.93  16.93  16.93  2550000.0      16.93
16586 1950-01-06  16.98  16.98  16.98  16.98  2010000.0      16.98
16585 1950-01-09  17.08  17.08  17.08  17.08  2520000.0      17.08


### Pulling useful features for prediction

We'll start by extracting some new aggregate data from our dataset which we can use for prediction. The three metrics we'll look at initially are average closing price, standard deviation of closing price, and the ratio of the standard deviations.

We need to create each of these new columns ourselves from the data that already exists in the dataset. For the closing price calculation for each row, we'll take the mean from the previous n-rows, where n is the number of days whose prices we're looking to average. To do this, we'll need to use the rolling, mean, and shift methods. We'll do something similar for the standard deviations.

We'll take the average closing price of the past 5, 30, and 180 trading days for each row, and we'll take the standard deviation of the past 5, and 30 closing prices. This way, we'll be able to see whether the number of days we consider when aggregating these summary statistics makes a difference to our model.

N.B. There are issues with the data we've selected, but we'll discuss this in more detail later one when we can see the results of our model.

In [102]:
data["avg_5"] = data["Close"].rolling(5).mean().shift()
data["avg_30"] = data["Close"].rolling(30).mean().shift()
data["avg_180"] = data["Close"].rolling(180).mean().shift()
data["std_5"] = data["Close"].rolling(5).std().shift()
data["std_30"] = data["Close"].rolling(30).std().shift()
data["rat_5_30"] = data["std_5"] / data["std_30"]

Let's make sure we remove all of the rows which contain null values. We'll do this by first removing all rows that fall within the first 180 trading days of our dataset (as these will have null values for avg_180) and then we'll remove any other rows with null values using the dropna method.

In [103]:
data_180 = data[data["Date"] > datetime(year = 1950, month = 9, day = 19)]
data_180 = data.dropna(axis = 0)

Our data is now ready to use for training a model. Let's split the data into train and test datasets. As we're trying to predict stock prices for dates from 2013 onwards, we'll partition our data based on date rather than proportion of the dataset.

In [104]:
train_1 = data_180[data_180["Date"] < datetime(year = 2013, month = 1, day = 1)]
test_1 = data_180[data_180["Date"] > datetime(year = 2012, month = 12, day = 31)]

### Creating and evaluating our models

Now we can create models to make predictions on our test data. We'll define a function which will take as an input our train and test sets and a list of features and then create a linear regression model and evaluate its accuracy. We'll measure the accuracy of the predictions of each model by taking the mean absolute error, an appropriate error metric given that we're trying to find out how accurate our model is in real terms (that is, how far off the closing price our model's prediction is in dollars).

We'll iterate through combinations of our features to work out which combination of features gives the best prediction. We can create this combinations with the itertools.combinations method.

In [105]:
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from itertools import combinations

maes = {}

def train_and_test(train, test, features):
    target = "Close"
    lr = LinearRegression()
    lr.fit(train[features], train[target])
    predictions = lr.predict(test[features])
    pred_mae = mean_absolute_error(predictions, test[target])
    
    return(pred_mae)

features = ["avg_5", "avg_30", "avg_180", "std_5", "std_30", "rat_5_30"]

feature_combos = []

for i in range(1, 7):
    for item in combinations(features, i):
        item = list(item)
        feature_combos.append(item)

maes = {}

for combo in feature_combos:
    mae = train_and_test(train, test, combo)
    key = "+".join(combo)
    maes[key] = mae

sorted_maes = sorted(maes.items(), key=lambda x: x[1])
sorted_maes

[('avg_5+avg_180+std_5', 16.109322393877076),
 ('avg_5+avg_180+std_5+rat_5_30', 16.109881009817116),
 ('avg_5+std_5', 16.118097517841054),
 ('avg_5+std_5+rat_5_30', 16.118206110794706),
 ('avg_5+avg_30+std_5', 16.126149929927873),
 ('avg_5+avg_30+std_5+rat_5_30', 16.127655827910925),
 ('avg_5+avg_30+avg_180+std_5', 16.130017941528674),
 ('avg_5+avg_30+avg_180+std_5+rat_5_30', 16.130382874836958),
 ('avg_5+avg_30+rat_5_30', 16.14820963274241),
 ('avg_5+avg_30', 16.14894968994607),
 ('avg_5+avg_30+avg_180+rat_5_30', 16.151439608957965),
 ('avg_5+avg_30+avg_180', 16.152758709969792),
 ('avg_5+avg_180+rat_5_30', 16.154107604008534),
 ('avg_5+std_5+std_30', 16.16228984561704),
 ('avg_5+avg_180', 16.171146983002608),
 ('avg_5+std_5+std_30+rat_5_30', 16.173632455238458),
 ('avg_5+avg_180+std_5+std_30', 16.181280773938514),
 ('avg_5+avg_180+std_5+std_30+rat_5_30', 16.1924622569356),
 ('avg_5+avg_180+std_30+rat_5_30', 16.204850946933654),
 ('avg_5+std_30+rat_5_30', 16.211312020224398),
 ('avg_5

### Primary analysis

The best model, juding solely by the model's MAE, was that which predicted the closing price based on the 5-day average, the 180-day average, and the 5-day standard deviation. However, if we look more closely at the differences in accuracy between our models, we can see that the difference is very small.

While in the world of stock trading differences in accuracy of a few cents can make a big difference to how much money we can make from trading, here it's interesting to note that most features have a minimal impact on our model's ability to predict closing price.

In fact, the only features which seem to have much impact at all are the average closing price features. In addition, the 30-day and 180-day averages make almost no difference to those models which use the 5-day average feature! Models which didn't use an average closing price feature at all were staggeringly inaccurate - with the best having a MAE of 883, and the worst a MAE of 1443.9!

### Improving the model

The reason for these results is quite obvious. The opening price on any given day is completely dependent on the closing price of the previous day. As such, the closing price on the given day will be highly dependant on the closing price of the previous day. No wonder aggregating prior closing prices led to better predictions.

It is also no surprise that a model which aggregated closing prices of days that were more recent performed better. While stock prices will be somewhat affected by events which occurred in the past, in general a stock price will change based on what happens on any given day. That is, prior closing prices are not relevant to the current closing price of a stock except for the fact that they constrain the starting value for a stock on a particular day. As such, averages which include stock prices from 180 days ago are going to be including a significant amount of data that is mostly irrelevant to the current closing price. We will therefore want to minimize the irrelevant price history we use to predict the current closing price.

As for the other features, they seemed to have very little impact. But, many of the models that used the 5-day standard deviation outperformed (albeit marginally) those that did not. What are we to make of this?

5-day standard deviation will measure the variation in the prices for the previous 5 days. If the price has been stable, std. dev. will be low, and if not, it will be high. How will knowing the std. dev. help us predict price? Stocks with high price std. dev. might be more likely to fluctuate, but how would we derive a price prediction just from knowing that a price fluctuation is more likely? If price has been increasing rapidly, and std. dev. is therefore high, we might predict a higher increase, or vice versa. But this would not be based on std. dev. alone but rather on rate of change and std. dev. But a model as simple as linear regression doesn't account for correlations between features, so it's not much use to us here.

Thus, moving forward we need only use 5-day average as a feature in our model. We might even consider using 1-4 day averages instead of 5-day average. Given the way stock prices work, we might expect the closing price on the previous day to be the best indicator of the closing price on a particular day. Let's try this out now.

### Optimizing the day-averages

We first need to add new averages to our dataset, and then we'll rerun our model to see which is best.

In [113]:
data["avg_4"] = data["Close"].rolling(4).mean().shift()
data["avg_3"] = data["Close"].rolling(3).mean().shift()
data["avg_2"] = data["Close"].rolling(2).mean().shift()
data["avg_1"] = data["Close"].rolling(1).mean().shift()

data_5 = data[data["Date"] > datetime(year = 1950, month = 1, day = 9)]
data_5 = data_5.dropna(axis = 0)

train_2 = data_5[data_5["Date"] < datetime(year = 2013, month = 1, day = 1)]
test_2 = data_5[data_5["Date"] > datetime(year = 2012, month = 12, day = 31)]

def univariate_train_and_test(train, test, feature):
    target = "Close"
    lr = LinearRegression()
    lr.fit(train[[feature]], train[target])
    predictions = lr.predict(test[[feature]])
    pred_mae = mean_absolute_error(predictions, test[target])
    
    return(pred_mae)

days = ["avg_4", "avg_3", "avg_2", "avg_1"]

day_maes = {}

for day in days:
    mae = univariate_train_and_test(train_2, test_2, day)
    day_maes[day] = mae

sorted_day_maes = sorted(day_maes.items(), key=lambda x: x[1])
sorted_day_maes

[('avg_1', 11.011952182180515),
 ('avg_2', 12.270448349909312),
 ('avg_3', 13.670676139088341),
 ('avg_4', 15.093564135148323)]

As expected, the fewer prices are taken into account, the more accurate the price prediction is.

### Adding additional features

The final thing we can change to improve our model is to add some additional features beyond what we considered above. If we look at the contents of our original dataset, volume looks like a metric that might be helpful in predicting price. Volume indicates the number of shares that were traded on a particular day. While this doesn't necessarily tell us whether prices are likely to go up or down, the more stocks that are traded (the higher the volume), the more likely a change in the stock price is.

It should be noted, however, that, as with standard deviation, volume alone gives no indication on the direction of change. It would need to be combined with something that indicates this before it can really be of use, and we cannot do this with a linear regression model. As such, we should not expect the volume traded to have much impact on our model.

As our final exercise, let's calculate average volume for 1, 5, and 30 days. We'll then use each of these features in our model, along with varying the number of n-day average prices, to see what gives us the best predictions.

In [126]:
data["vol_1"] = data["Volume"].rolling(1).mean().shift()
data["vol_5"] = data["Volume"].rolling(5).mean().shift()
data["vol_30"] = data["Volume"].rolling(30).mean().shift()

data_30 = data[data["Date"] > datetime(year = 1950, month = 2, day = 14)]
data_30 = data_30.dropna(axis = 0)

train_3 = data_30[data_30["Date"] < datetime(year = 2013, month = 1, day = 1)]
test_3 = data_30[data_30["Date"] > datetime(year = 2012, month = 12, day = 31)]

days = ["avg_5", "avg_4", "avg_3", "avg_2", "avg_1"]
volumes = ["vol_1", "vol_5", "vol_30"]

from itertools import product

combs = product(days, volumes)

day_vols = []

for item in combs:
    item = list(item)
    day_vols.append(item)

day_vol_maes = {}

for com in day_vols:
    mae = train_and_test(train_3, test_3, com)
    key = "_".join(com)
    day_vol_maes[key] = mae
    
sorted_day_vol_maes = sorted(day_vol_maes.items(), key=lambda x: x[1])
sorted_day_vol_maes

[('avg_1_vol_1', 11.012102639779847),
 ('avg_1_vol_30', 11.012140633297475),
 ('avg_1_vol_5', 11.013157176186834),
 ('avg_2_vol_1', 12.268451338424754),
 ('avg_2_vol_30', 12.269958355881325),
 ('avg_2_vol_5', 12.270379377948652),
 ('avg_3_vol_1', 13.65665728155126),
 ('avg_3_vol_5', 13.667788272447675),
 ('avg_3_vol_30', 13.670182005607735),
 ('avg_4_vol_1', 15.065431688977933),
 ('avg_4_vol_5', 15.089839528750804),
 ('avg_4_vol_30', 15.094217980968926),
 ('avg_5_vol_1', 16.231328652220462),
 ('avg_5_vol_5', 16.26449238111367),
 ('avg_5_vol_30', 16.270684615130566)]

As expected, volume has very little impact on a model's accuracy (although, if we're splitting hairs, the volume of trades on the previous day seems to be the best indicator).

### Conclusion

Our most successful model was that which used only the 1-day average price to predict a future day's closing price.

However, there were other improvements we could have made to our model. 

Firstly, if we'd chosen an algorithm that could have considered relations between features, we may have been able to make better use of the data we had on volume and standard deviation. Our linear regression model couldn't do this.

Secondly, we could have evaluated how the price had changed in previous days rather than just considering the average of those prices. If a stock has been decreasing in value for 5 straight days, then this may be an indicator of a decrease in value on day 6 as well.

Finally, our current data is on aggregate stock prices and so does not look at specific businesses, but it need not be. If it were on specific businesses, we could have incorporated additional data into our model relating to recent events regarding specific businesses. For example, if a CEO was replaced, or the company announced a plan for expansion into a new state, and so on.